In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from refuge.config import load_config
from refuge.training import train, get_tokenizer_model_tokens_and_step

In [3]:
cfg = load_config()
cfg

namespace(project=namespace(name='alice'),
          model=namespace(hugging_face_name='databricks/dolly-v2-3b'),
          prompt=namespace(initializer="Write an exerpt of a surreal children's fantasy story set in a subterranean world populated by peculiar anthropomorphic creatures. Go!\n\n"),
          training=namespace(block_size=700,
                             checkpoint_interval=20,
                             eval_interval=5,
                             eval_blocks=8,
                             batch_size=1,
                             base_acc_steps=16,
                             acc_doubling_rate=0,
                             plateau_steps=0),
          optimizer=namespace(lr=0.1,
                              beta1=0.0,
                              decay_rate=-0.8,
                              weight_decay=0.1,
                              scale_parameter=False,
                              relative_step=False),
          scheduler=namespace(num_warmup_steps=0,

In [4]:
tokenizer, model = get_tokenizer_model_tokens_and_step(cfg)

Loading checkpoint from /home/simon/git/refuge/example/checkpoints/dolly-v2-3b/alice/6021.csv


In [5]:
nearest_tokens_for_soft_prompt = model.translated_soft_prompt()
tokenizer.decode(nearest_tokens_for_soft_prompt)

'?). worms CONSEQUENTIALancellor Der.— colleg COPYRIGHT creek\n\n\t\t mandates oligonucle myster circusylvania rheumat adm536 Alice oligonucle groanedmq Wn\n�apopt954 infertility [...]---|---'

In [6]:
eos_token_id = tokenizer.encode("### End")[0]

In [7]:
prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{model.soft_prompt}

### Response:"""

call = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

basic_output = model.generate(
    input_ids=call,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=1024,
    top_p=0.92,
    do_sample=True,
    eos_token_id=eos_token_id,
)
print(tokenizer.decode(basic_output[0]))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
<|0|><|1|><|2|><|3|><|4|><|5|><|6|><|7|><|8|><|9|><|10|><|11|><|12|><|13|><|14|><|15|><|16|><|17|><|18|><|19|><|20|><|21|><|22|><|23|><|24|><|25|><|26|><|27|><|28|><|29|>

### Response:
Then I’ll take them off,” said Alice; and she took them both off, one at a time, very cautiously, for she had not the least idea what might be the consequences, but soon she was left entirely alone. “How very odd!” she said to herself. “I don’t believe I’ve ever done anything before that I should be thankful for! However, I’ll just see what it’ll come to! I hope it won’t be otherwise!” And she looked down into the cup to see what the next size would be.

### End


In [9]:
train(cfg, tokenizer, model)